In [1]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.models import load_model
from keras import backend as K
import numpy as np
from keras.layers.normalization import BatchNormalization
import nbimporter
from Train_Dataset_Generation import *
import pydot
import graphviz

Using Theano backend.


Importing Jupyter notebook from Train_Dataset_Generation.ipynb
Importing Jupyter notebook from Dataset_Preparation.ipynb


### Hyperparameters

In [2]:
n_classes = 133
n_rows = 32
n_cols = 32
batch_size = 16
epochs = 1
learning_rate = 0.1
decay = 1e-6
momentum = .9

### Creating The Model

In [3]:
def create_model():
    # conv-conv-pool ==> conv-conv-pool ==> dense ==>dense
    print('Building Model..')
    model = Sequential()
    
    model.add(Conv2D(32,(3,3),input_shape=(n_rows,n_cols,1),activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    BatchNormalization(axis=1)
    model.add(Conv2D(64,(3,3),activation='relu'))
    BatchNormalization(axis=1)
    model.add(MaxPooling2D((2,2),strides=(2,2)))
    model.add(ZeroPadding2D((1,1)))
    
    model.add(Conv2D(64,(3,3),activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    BatchNormalization(axis=1)
    model.add(Conv2D(64,(3,3),activation='relu')) 
    BatchNormalization(axis=1)

    model.add(MaxPooling2D((2,2),strides=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense(2018, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(n_classes, activation='softmax'))
    
    sgd = SGD(lr = learning_rate, decay = decay, momentum = momentum,nesterov=True)
    model.compile(optimizer = sgd, loss='categorical_crossentropy',metrics=['accuracy'])
    
    print('Model Generated')
    
    return model

In [4]:
def trainCNN(X_train,y_train):
    
    model = create_model()
    print('Training Started . Please Wait ......')
    model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, verbose = 1)
    print('Training Completed')
    
    return model

In [5]:
def trainCNNWithGenerator(dataGen,X,y):
    model = create_model()
    print('Training Started . Please Wait ......')
    model.fit_generator(
        dataGen.flow(X,y,batch_size = 256),
        steps_per_epoch=256,
        epochs=20)
    print('Training Completed')
    model.save_weights('first_try.h5')
    return model

In [6]:
def getGenerator():
    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        fill_mode='nearest')
    return datagen

In [7]:
def saveWeights(model,name):
    model.save('Models/'+name+'.h5')
    print('Model Saved to Models/'+name+'.h5')

In [8]:
def loadSavedWeight(name):
    model = load_model('Models/'+name+'.h5')
    return model

In [9]:
def MalayalamCharacterRecognition():

    X_train,X_test,y_train,y_test = mal_char_data()
    print('Loaded Dataset')
    model = trainCNN(X_train,y_train)
    saveWeights(model,"train1")
    return model

In [10]:
def MalCharRecog_BN_Aug():
    
    X_train,X_test,y_train,y_test = mal_char_data()
    trainGen = getGenerator()
    model = trainCNNWithGenerator(trainGen,X_train,y_train)
    return model

In [13]:
model = MalayalamCharacterRecognition()

Loaded Dataset
Building Model..
Model Generated
Training Started . Please Wait ......


KeyboardInterrupt: 

In [11]:
model = create_model()

Building Model..
Model Generated


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        320       
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        36928     
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 17, 17, 64)        0         
__________

In [16]:
model.load_weights("/home/amrith/Machine-Learning/MalayalamOCR/Successful-Models/train3_bn_96.h5")

In [12]:
from theano import *
convout1_f = theano.function([model.get_input(train=False)], convout1.get_output(train=False))

AttributeError: 'Sequential' object has no attribute 'get_input'

In [14]:
import keras.utils.vis_utils

In [13]:
from keras.utils import plot_model
# plot_model(model, to_file='model.png')

In [14]:
plot_model(model, to_file='model.png')